In [2]:
import os
import pandas as pd
import sys
from collections import defaultdict
import numpy as np
import time
import datetime
import re
import shutil

## 인자값 입력

### 1) 전처리할 파일들이 있는 디렉토리 경로
### 4) 출력파일 디렉토리 경로

In [3]:
# 데이터 파일 디렉토리 경로
data_path = './data'
# 출력 파일 경로
out_path = './out'

if data_path[-1] == '/':
    data_path = data_path[:-1]
if out_path[-1] == '/':
    out_path = out_path[:-1]

## (사용자 설정 디렉토리 내) 전처리대상 파일 리스트 자동 추출

In [4]:
def recursive_search_dir(_nowDir, _filelist):
    dir_list = []  # 현재 디렉토리의 서브디렉토리가 담길 list
    f_list = os.listdir(_nowDir)
    for fname in f_list:
        if fname == "checked_files" or fname == "result_files":  # 이미 전처리한 폴더는 통과
            continue
        if os.path.isdir(_nowDir + "/" + fname):
            dir_list.append(_nowDir + "/" + fname)
        elif os.path.isfile(_nowDir + "/" + fname):
            file_extension = os.path.splitext(fname)[1]
            if file_extension == ".csv" or file_extension == ".CSV":  # csv
                _filelist.append(_nowDir + "/" + fname)

    for toDir in dir_list:
        recursive_search_dir(toDir, _filelist)
        
csv_list = []
print('CSV 파일 목록 불러오는 중..')
recursive_search_dir(data_path, csv_list)
print('총 CSV파일 수 : {}'.format(len(csv_list)))

CSV 파일 목록 불러오는 중..
총 CSV파일 수 : 10


In [5]:
pd.read_csv(csv_list[0], low_memory=False)

,ins_dt,dev_id,coll_dt,coll_hour,is_snd,snd_dt,car_id,b_soc,b_pack_current,b_pack_volt,...,b_cell172_volt,b_cell173_volt,b_cell174_volt,b_cell175_volt,b_cell176_volt,b_cell177_volt,b_cell178_volt,b_cell179_volt,b_cell180_volt,car_type
0,2022-01-14 14:16:00.890,M2210703775,2022-01-14 14:04:57.000,2022011414,True,2022-01-14 15:01:39.233,NaN,75.0,0.4,704.6,...,3.90,3.90,3.90,3.90,3.90,3.90,3.90,3.90,3.90,IONIQ5
1,2022-01-14 14:16:00.890,M2210703775,2022-01-14 14:05:07.000,2022011414,True,2022-01-14 15:01:39.233,NaN,75.0,0.5,704.6,...,3.90,3.90,3.90,3.90,3.90,3.90,3.90,3.90,3.90,IONIQ5
2,2022-01-14 14:16:00.890,M2210703775,2022-01-14 14:05:17.000,2022011414,True,2022-01-14 15:01:39.233,NaN,75.0,0.4,704.6,...,3.90,3.90,3.90,3.90,3.90,3.90,3.90,3.90,3.90,IONIQ5
3,2022-01-14 14:16:00.890,M2210703775,2022-01-14 14:05:28.000,2022011414,True,2022-01-14 15:01:39.233,NaN,75.0,0.3,704.6,...,3.90,3.90,3.90,3.90,3.90,3.90,3.90,3.90,3.90,IONIQ5
4,2022-01-14 14:16:00.890,M2210703775,2022-01-14 14:05:38.000,2022011414,True,2022-01-14 15:01:39.233,NaN,75.0,0.5,704.6,...,3.90,3.90,3.90,3.90,3.90,3.90,3.90,3.90,3.90,IONIQ5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2994,2022-01-20 00:15:00.817,M2210703775,2022-01-20 00:04:15.000,2022012000,True,2022-01-20 01:01:32.643,NaN,62.0,-34.2,693.4,...,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,IONIQ5
2995,2022-01-20 00:15:00.817,M2210703775,2022-01-20 00:04:25.000,2022012000,True,2022-01-20 01:01:32.643,NaN,62.0,-34.4,693.4,...,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,IONIQ5
2996,2022-01-20 00:15:00.817,M2210703775,2022-01-20 00:04:35.000,2022012000,True,2022-01-20 01:01:32.643,NaN,62.0,-34.3,693.5,...,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,IONIQ5
2997,2022-01-20 00:15:00.817,M2210703775,2022-01-20 00:04:45.000,2022012000,True,2022-01-20 01:01:32.643,NaN,62.5,-34.3,693.5,...,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,3.84,IONIQ5


## 데이터 전처리 및 규격화

In [16]:
def printProgressBar(iteration, total, prefix = 'Progress', suffix = 'Complete',\
                      decimals = 1, length = 50, fill = '█'): 
    # 작업의 진행상황을 표시
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r%s |%s| %s%% %s' %(prefix, bar, percent, suffix), end='\r')
    sys.stdout.flush()
    if iteration == total:
        print()


def preprocessing(csv_file, csv_path, out_path):

    skip_rows = 0
    total_lines = 0
    batch_size = 50000  # 500,000으로 설정 시, 메모리 부족 오류 발생 (해당 서버 정보: 코어개수 16, Ram: 8G, 사용 CPU 수: 5)
    isError=False
    cols = []
    while True:
        try:
            if len(cols) == 0:
                df = pd.read_csv(csv_file, low_memory=False, skiprows=skip_rows, nrows=batch_size)
            else:
                df = pd.read_csv(csv_file, low_memory=False, names=cols, header=None, skiprows=skip_rows, nrows=batch_size)

            for field in df.columns:
                if 'Unnamed: ' in field:
                    df.drop(columns = field, inplace=True)
        except Exception as e:
            if str(e).startswith('No columns to parse from file'):
                break
            else:
                print('\n')
                print(csv_file)
                print(e)
                isError=True
                break

        if len(df) == 0:
            break
        else:
            total_lines += len(df)
            skip_rows += batch_size

            # 필드명을 모두 소문자로 변환 (사유: 22년 6월 데이터부터, 필드명이 대문자로 바뀜. 이전 데이터와 통일하기 위해.)
            df.rename(mapper=str.lower, axis='columns', inplace=True)
        try:
            df.sort_values(by='coll_dt', inplace=True)
            df.reset_index(drop=True, inplace=True)
        except Exception as e:
            print('\n')
            print(csv_file)
            print(e)
            isError=True
            break
        cols = list(df.columns)

        # 날짜, 시간 형식에서 'T', 'Z' 문자 제거 ('T', 'Z' 포함되는 것은 ISO 8601 형태이며, 데이터 통일을 위해 문자 제거)
        df['coll_dt'] = df['coll_dt'].str.replace('T', ' ').str.replace('Z', '')

        for i in range(len(df)):
            df_curr_row = df.iloc[i]

            # 오류 값 처리
            for field in df.columns:
                val = df_curr_row.loc[field]
        
                try:
                    val = float(val)
                except:
                    continue

                # soc 필드인 경우, 값이 0~100을 벗어나면 오류값이므로 NaN값으로 변경
                if 'soc' in field:
                    if val < 0 or val > 100:
                        df.loc[df.index[i], field] = np.nan
                    continue

                # 전압 필드인 경우, 값이 3,000 초과이면 오류값으로 판단 NaN값으로 변경
                if '_volt' in field:
                    if val > 3000:
                        df.loc[df.index[i], field] = np.nan
                    continue

                # 온도 필드인 경우, 값이 -35 미만 또는 +80 초과이면 오류값으로 판단 NaN값으로 변경
                if '_temp' in field:
                    if val < -35 or val > 80:
                        df.loc[df.index[i], field] = np.nan
                    continue

                # 전류 필드인 경우, 값이 -500 미만 또는 +500 초과이면 오류값으로 판단 NaN값으로 변경
                if 'b_pack_current' in field:
                    if val < -500 or val > 500:
                        df.loc[df.index[i], field] = np.nan
                    continue

                # 차량 속도 필드인 경우, 값이 0 미만 또는 180 초과이면 오류값으로 판단 NaN값으로 변경
                if 'car_speed' in field:
                    if val < 0 or val > 180:
                        df.loc[df.index[i], field] = np.nan
                    continue

                # 배터리 전압 필드인 경우, 값이 0 미만 또는 6 초과이면 오류값으로 판단 NaN값으로 변경
                if field.startswith('b_cell') and field.endswith('_volt'):
                    if val < 1 or val > 6:
                        df.loc[df.index[i], field] = np.nan
                    continue

                # 누적 관련 양 값들이 1,000,000 초과이면 오류값으로 판단 NaN값으로 변경
                if 'b_accum' in field:
                    if val > 1000000:
                        df.loc[df.index[i], field] = np.nan
                    continue

                # 누적주행거리 필드인 경우, 값이 0 이하 또는 2,000,000 초과이면 오류값으로 판단 NaN값으로 변경
                if 'mileage' in field:
                    if val <= 0 or val > 2000000:
                        df.loc[df.index[i], field] = np.nan

        # csv 파일에 저장
        subpath = '/'.join(list((csv_file.split(csv_path)[-1]).split('/'))[:-1])
        if len(subpath) == 0:
            subpath = '.'
        result_path = out_path + '/' + subpath + '/'
        if not os.path.isdir(result_path):
            os.makedirs(result_path)
        file_name = csv_file.split('/')[-1]
        if os.path.isfile(result_path + file_name):
            df.to_csv(result_path + file_name, mode='a', header=False, index=False)
        else:
            df.to_csv(result_path + file_name, mode='w', header=True, index=False)
    return total_lines



print('\nCSV 파일 2차 전처리 중..')
cnt=0
for csv_file in csv_list:
    printProgressBar(cnt, len(csv_list))
    preprocessing(csv_file, data_path, out_path)
    cnt+=1
printProgressBar(cnt, len(csv_list))



CSV 파일 2차 전처리 중..
Progress |██████████████████████████████████████████████████| 100.0% Complete


## 출력 파일 확인

In [ ]:
csv_list = []
print('CSV 파일 목록 불러오는 중..')
recursive_search_dir(out_path, csv_list)
csv_list